<a href="https://colab.research.google.com/github/kashimmirza/K-Learning/blob/master/customizedautoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
from datetime import datetime
import pandas as pd
import random 
from torchvision.datasets import ImageFolder
import re
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
from skimage.io import imread, imsave
import skimage
from PIL import ImageFile
from PIL import Image
from torchsummary import summary

torch.cuda.empty_cache()

In [ ]:
# normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

########## Mean and std are calculated from the train dataset
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])
train_transformer = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
    transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(90),
    # random brightness and random contrast
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    normalize
])

val_transformer = transforms.Compose([
#     transforms.Resize(224),
#     transforms.CenterCrop(224),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    normalize
])

In [1]:
batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):
        """
        Args:
            txt_path (string): Path to the txt file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        File structure:
        - root_dir
            - CT_COVID
                - img1.png
                - img2.png
                - ......
            - CT_NonCOVID
                - img1.png
                - img2.png
                - ......
        """
        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        sample = {'img': image,
                  'label': int(self.img_list[idx][1])}
        return sample


NameError: ignored

In [ ]:
if __name__ == '__main__':
    trainset = CovidCTDataset(root_dir='E:/pythonlatestfolder1/datasets/new_data/4.4image',
                              txt_COVID='E:/pythonlatestfolder1/datasets/new_data/newtxt/train.txt',
                              txt_NonCOVID='E:/pythonlatestfolder1/datasets/old_data/oldtxt/trainctnoncovid.txt',
                              transform= train_transformer)
    valset = CovidCTDataset(root_dir='E:/pythonlatestfolder1/datasets/new_data/4.4image',
                              txt_COVID='E:/pythonlatestfolder1/datasets/new_data/newtxt/val.txt',
                              txt_NonCOVID='E:/pythonlatestfolder1/datasets/old_data/oldtxt/valctnoncovid.txt',
                              transform= val_transformer)
    testset = CovidCTDataset(root_dir='E:/pythonlatestfolder1/datasets/new_data/4.4image',
                              txt_COVID='E:/pythonlatestfolder1/datasets/new_data/newtxt/test.txt',
                              txt_NonCOVID='E:/pythonlatestfolder1/datasets/old_data/oldtxt/testct_noncovid.txt',
                              transform= val_transformer)
    print(trainset.__len__())
    print(valset.__len__())
    print(testset.__len__())

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=False)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)

FileNotFoundError: ignored

In [ ]:
for batch_index, batch_samples in enumerate(train_loader):      
        data, target = batch_samples['img'], batch_samples['label']
skimage.io.imshow(data[0,1,:,:].numpy())

NameError: ignored

In [ ]:
print(data.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(3, 16, 3, padding =0)
        self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        self.pool = nn.MaxPool2d(2, 2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 3, 2, stride=2)

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)# compressed representation
        encoder = x;
        m = nn.flatten()
        encoder = m(encoder)
        print('encoder :'encoder)
        
        print(x)
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
model = model.to(device)
print(model)
summary(model,(3,224,224))

SyntaxError: ignored

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(3, 512, 3, padding =0,stride =2)
        self.conv1 = nn.Conv2d(512, 64, 3, padding =0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        
        self.conv2 = nn.Conv2d(256, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 256, 3, stride=2)
        self.t_conv1 = nn.Conv2d(3, 64, 3, padding =0,stride =2)
        self.conv1 = nn.Conv2d(64, 64, 3, padding =0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)# compressed representation
        encoder = x;
        
        print(x)
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv2(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
model = model.to(device)
print(model)
summary(model,(3,224,224))

ConvAutoencoder(
  (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
  (t_conv2): ConvTranspose2d(512, 256, kernel_size=(3, 3), stride=(2, 2))
  (t_conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2))
)


RuntimeError: ignored

In [4]:
# final customized cnn model by calculating
import torch.nn as nn
import torch.nn.functional as F
y=15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), 3x3 kernels
        self.conv1 = nn.Conv2d(3, 512, 5, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(512, 256, 5, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(256, 128, 2, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(128,64, 3, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv5 = nn.Conv2d(64,32, 3, padding =0,stride =1)
        
        self.pool = nn.MaxPool2d(2,2)
        self.classifier = nn.Linear(64*32*3, 2)
        classifier = self.classifier
        print('classifier',classifier)
        
        
        
        
         ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(32, 64, 3, padding=0, stride=1)
        self.t_conv2 = nn.ConvTranspose2d(64, 128, 3,padding=0, stride=1)
        self.t_conv3 = nn.ConvTranspose2d(128, 256, 2,padding=0, stride=1)
        self.t_conv4 = nn.ConvTranspose2d(256, 512, 5,padding=0, stride=1)
        self.t_conv5 = nn.ConvTranspose2d(512, 256, 5,padding=0, stride=1)
        
        
        #self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        '''self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        '''
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        
        '''self.conv2 = nn.Conv2d(256, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 256, 3, stride=2)
        self.t_conv1 = nn.Conv2d(3, 64, 3, padding =0,stride =2)
        self.conv1 = nn.Conv2d(64, 64, 3, padding =0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        '''

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # add third hidden layer
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # add fouth hidden layer
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        # add 5th hidden layer
        x = F.relu(self.conv5(x))
        x = self.pool(x)# compressed representation
        encoder = x;
        output =nn.Flatten();
        output =output(encoder)
        global y
        y=output

        
        print(x)
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.relu(self.t_conv3(x))
        x = F.relu(self.t_conv4(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv5(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
model = model.to(device)
print(model)
summary(model,(3,224,224))

classifier Linear(in_features=6144, out_features=2, bias=True)
ConvAutoencoder(
  (conv1): Conv2d(3, 512, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(512, 256, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(256, 128, kernel_size=(2, 2), stride=(1, 1))
  (conv4): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (classifier): Linear(in_features=6144, out_features=2, bias=True)
  (t_conv1): ConvTranspose2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (t_conv2): ConvTranspose2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (t_conv3): ConvTranspose2d(128, 256, kernel_size=(2, 2), stride=(1, 1))
  (t_conv4): ConvTranspose2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
  (t_conv5): ConvTranspose2d(512, 256, kernel_size=(5, 5), stride=(1, 1))
)
tensor([[[[0.0120, 0.0118, 0.0113, 0.0117, 0.0120],
          [0.0126, 0.0127, 0.0117, 0

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [6]:
print(y)
print('size',y.size())
k=torch.flatten(y)
#k=k(y)
print(k)
#k=2*800
print(k)
y.size()
        

tensor([[0.0120, 0.0118, 0.0113,  ..., 0.0000, 0.0000, 0.0000],
        [0.0122, 0.0119, 0.0115,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReshapeAliasBackward0>)
size torch.Size([2, 800])
tensor([0.0120, 0.0118, 0.0113,  ..., 0.0000, 0.0000, 0.0000],
       grad_fn=<ReshapeAliasBackward0>)
tensor([0.0120, 0.0118, 0.0113,  ..., 0.0000, 0.0000, 0.0000],
       grad_fn=<ReshapeAliasBackward0>)


torch.Size([2, 800])

In [ ]:
#customized cnn model by calculating for 32 X32
import torch.nn as nn
import torch.nn.functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## encoder layers ##
        # conv layer (depth from 3 --> 16), 3x3 kernels
        x = nn.Conv2d(3, 16,3 , padding =0,stride =1)
        #x= nn.ReLU(x)
        x = nn.MaxPool2d(2,2)
        x = nn.Conv2d(16, 8, 3, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(8, 4, 3, padding =0,stride =1)
        self.pool = nn.MaxPool2d(2,2)
        
        
        
        
        
         ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv1 = nn.ConvTranspose2d(32, 64, 3, padding=0, stride=1)
        self.t_conv2 = nn.ConvTranspose2d(64, 128, 3,padding=0, stride=1)
        self.t_conv3 = nn.ConvTranspose2d(128, 256, 2,padding=0, stride=1)
        self.t_conv4 = nn.ConvTranspose2d(256, 512, 5,padding=0, stride=1)
        self.t_conv5 = nn.ConvTranspose2d(512, 256, 5,padding=0, stride=1)
        
        
        #self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        '''self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        '''
        # pooling layer to reduce x-y dims by two; kernel and stride of 2
        
        '''self.conv2 = nn.Conv2d(256, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(512, 512, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        
        ## decoder layers ##
        ## a kernel of 2 and a stride of 2 will increase the spatial dims by 2
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 512, 3, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(512, 256, 3, stride=2)
        self.t_conv1 = nn.Conv2d(3, 64, 3, padding =0,stride =2)
        self.conv1 = nn.Conv2d(64, 64, 3, padding =0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        # conv layer (depth from 16 --> 4), 3x3 kernels
        self.conv2 = nn.Conv2d(64, 256, 3, padding=0,stride =2)
        self.conv2 = nn.Conv2d(256, 256, 3, padding=0,stride =2)
        self.pool = nn.MaxPool2d(2,2)
        '''

    def forward(self, x):
        ## encode ##
        # add hidden layers with relu activation function
        # and maxpooling after
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        # add second hidden layer
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        # add third hidden layer
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        # add fouth hidden layer
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        # add 5th hidden layer
        x = F.relu(self.conv5(x))
        x = self.pool(x)# compressed representation
        encoder = x;
        
        print(x)
        
        ## decode ##
        # add transpose conv layers, with relu activation function
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.relu(self.t_conv3(x))
        x = F.relu(self.t_conv4(x))
        # output layer (with sigmoid for scaling from 0 to 1)
        x = F.sigmoid(self.t_conv5(x))
                
        return x

# initialize the NN
model = ConvAutoencoder()
model = model.to(device)
print(model)
summary(model,(3,224,224))

NameError: ignored

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import glob
import cv2
import pickle

from keras.models import Sequential
from keras.layers import Conv2D
import os

print(os.listdir("images/"))

SIZE = 512 #Resize images

train_images = []

for directory_path in glob.glob("images/train_images"):
    for img_path in glob.glob(os.path.join(directory_path, "*.tif")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        #train_labels.append(label)
        
train_images = np.array(train_images)

train_masks = [] 
for directory_path in glob.glob("images/train_masks"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE, SIZE))
        #mask = cv2.cvtColor(mask, cv2.COLOR_RGB2BGR)
        train_masks.append(mask)
        #train_labels.append(label)
        
train_masks = np.array(train_masks)

X_train = train_images
y_train = train_masks
y_train = np.expand_dims(y_train, axis=3)


activation = 'sigmoid'
feature_extractor = Sequential()
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', input_shape = (SIZE, SIZE, 3)))
feature_extractor.add(Conv2D(32, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))

#feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
#feature_extractor.add(BatchNormalization())
#
#feature_extractor.add(Conv2D(64, 3, activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
#feature_extractor.add(BatchNormalization())
#feature_extractor.add(MaxPooling2D())
#feature_extractor.add(Flatten())

X = feature_extractor.predict(X_train)

X = X.reshape(-1, X.shape[3])

Y = y_train.reshape(-1)

dataset = pd.DataFrame(X)
dataset['Label'] = Y
print(dataset['Label'].unique())
print(dataset['Label'].value_counts())

##If we do not want to include pixels with value 0 
##e.g. Sometimes unlabeled pixels may be given a value 0.
dataset = dataset[dataset['Label'] != 0]

X_for_RF = dataset.drop(labels = ['Label'], axis=1)
Y_for_RF = dataset['Label']

#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators = 50, random_state = 42)

# Train the model on training data
# Ravel Y to pass 1d array instead of column vector
model.fit(X_for_RF, Y_for_RF) #For sklearn no one hot encoding


filename = 'RF_model.sav'
pickle.dump(model, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))



#READ EXTERNAL IMAGE...
test_img = cv2.imread('images/test_images/Sandstone_Versa0360.tif', cv2.IMREAD_COLOR)       
test_img = cv2.resize(test_img, (SIZE, SIZE))
test_img = cv2.cvtColor(test_img, cv2.COLOR_RGB2BGR)
test_img = np.expand_dims(test_img, axis=0)

#predict_image = np.expand_dims(X_train[8,:,:,:], axis=0)
X_test_feature = feature_extractor.predict(test_img)
X_test_feature = X_test_feature.reshape(-1, X_test_feature.shape[3])

prediction = loaded_model.predict(X_test_feature)

prediction_image = prediction.reshape(mask.shape)
plt.imshow(prediction_image, cmap='gray')


In [ ]:
X_train_encode = encoder.predict(X_train)

In [ ]:
#tensorflow based modified simple auto encoder

#Importing the Libraries
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model

In [ ]:
# define dataset
X, y = make_classification(n_samples=1000, n_features=100, n_informative=10, n_redundant=90, random_state=1)
# summarize the dataset
print(X.shape, y.shape)

(1000, 100) (1000,)


In [ ]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# scale data
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)

In [ ]:
# AutoEncoder Model Preparation
n_inputs = X.shape[1]
# define encoder
input_data_shape= Input(shape=(n_inputs,))
# encoder level 1
encoder= Dense(n_inputs*2)(input_data_shape)
encoder = BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
# encoder level 2
encoder= Dense(n_inputs)(encoder)
encoder= BatchNormalization()(encoder)
encoder= LeakyReLU()(encoder)
print(encoder)
# bottleneck
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(encoder)
# define decoder, level 1
decoder = Dense(n_inputs)(bottleneck)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)
# decoder level 2
decoder = Dense(n_inputs*2)(decoder)
decoder = BatchNormalization()(decoder)
decoder = LeakyReLU()(decoder)

KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.float32, name=None), name='leaky_re_lu_1/LeakyRelu:0', description="created by layer 'leaky_re_lu_1'")


In [ ]:
# output layer
output = Dense(n_inputs, activation='linear')(decoder)
# define autoencoder model
model = Model(inputs=input_data_shape, outputs=output)
# compile autoencoder model
model.compile(optimizer='adam', loss='mse')

In [ ]:
# fit the autoencoder model to reconstruct input
# fit the autoencoder model to reconstruct input
history = model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=2, validation_data=(X_test,y_test))

In [ ]:
# define an encoder model (without the decoder)
encoder = Model(inputs=input_data_shape, outputs=bottleneck)
# save the encoder to file
encoder.save('encoder.h5')

In [ ]:
#Building a Base Model to compare the performance after compressing the data using Encoder model.

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
t = MinMaxScaler()
t.fit(X_train)
X_train = t.transform(X_train)
X_test = t.transform(X_test)
# define model
model = LogisticRegression()
# fit model on training set
model.fit(X_train, y_train)
# make prediction on test set
yhat = model.predict(X_test)
# calculate accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

In [ ]:
#Compressing the input data using Encoder Model and fitting it on the Logistic Regression model.
# load the model from file
encoder = load_model('encoder.h5')

# encode the train data
X_train_encode = encoder.predict(X_train)
# encode the test data
X_test_encode = encoder.predict(X_test)
# define the model
model = LogisticRegression(max_iter=1000)
# fit the model on the training set
model.fit(X_train_encode, y_train)
# make predictions on the test set
yhat = model.predict(X_test_encode)
# calculate classification accuracy
acc = accuracy_score(y_test, yhat)
print(acc)

OSError: ignored

# tutorial for cifer

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters 
num_epochs = 5
batch_size = 4
learning_rate = 0.001

# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# CIFAR10: 60000 32x32 color images in 10 classes, with 6000 images per class
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size,
                                         shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 512, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 14, 14
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 16, 5, 5
        x = x.view(-1, 16 * 5 * 5)            # -> n, 400
        x = F.relu(self.fc1(x))               # -> n, 120
        x = F.relu(self.fc2(x))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)

with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')